In [1]:
import dicom2nifti
import os 
import pydicom
import matplotlib.pyplot as plt
import SimpleITK as sitk
import numpy as np

In [17]:
# Load the RT Struct file
rt_struct_path = "/home/tachennf/Documents/delta-rad/madrid_study/data/Mont1/Montpellier_mont1_RTst_2024-07-11_110955_MRI.Study.from.2024-07-11.02.04_RTSTR_n1__00000/2.16.840.1.114362.1.12289667.27454039548.698351266.280.551.dcm"
rt_struct = pydicom.dcmread(rt_struct_path)

# # Access basic information
# print(f"Modality: {rt_struct.Modality}")
# print(f"Number of Structures: {len(rt_struct.StructureSetROISequence)}")
# print(f"Patient Name: {rt_struct.StructureSetROISequence}")
# 
# # Iterate over structures
# for roi in rt_struct.StructureSetROISequence:
#     print(f"ROI Name: {roi.ROIName}")
#     print(f"ROI Number: {roi.ROINumber}")

# Access contour data
for contour in rt_struct.ROIContourSequence:
    # print(f"Contour Geometric Type: {contour}")
    print(f"Number of Contour Points: {contour.ContourSequence[0].ContourData}")
    continue

Number of Contour Points: [13.4678, -62.9823, -130.7115, 14.9678, -62.9823, -130.7115, 16.4678, -62.9823, -130.7115, 17.9678, -62.9823, -130.7115, 19.4678, -62.9823, -130.7115, 20.9678, -62.9823, -130.7115, 22.4678, -62.9823, -130.7115, 23.9678, -62.9823, -130.7115, 25.4678, -62.9823, -130.7115, 26.9678, -62.9823, -130.7115, 28.4678, -62.9823, -130.7115, 29.9678, -62.9823, -130.7115, 31.4678, -62.9823, -130.7115, 32.9678, -62.9823, -130.7115, 34.4678, -62.9823, -130.7115, 35.9678, -62.9823, -130.7115, 37.4678, -62.9823, -130.7115, 38.9678, -62.9823, -130.7115, 40.4678, -62.9823, -130.7115, 41.9678, -62.9823, -130.7115, 43.4678, -62.9823, -130.7115, 44.9678, -62.9823, -130.7115, 46.4678, -62.9823, -130.7115, 47.9678, -62.9823, -130.7115, 49.4678, -62.9823, -130.7115, 50.9678, -62.9823, -130.7115, 52.4678, -62.9823, -130.7115, 53.9678, -62.9823, -130.7115, 55.4678, -62.9823, -130.7115, 55.4678, -62.9823, -130.7115, 56.9678, -61.4823, -130.7115, 58.4678, -61.4823, -130.7115, 59.9678, -61.

AttributeError: 'Dataset' object has no attribute 'ContourSequence'

In [2]:
reader = sitk.ImageFileReader()
reader.SetFileName("/home/tachennf/Documents/delta-rad/madrid_study/data/Mont1/Montpellier_mont1_RTst_2024-07-11_110955_MRI.Study.from.2024-07-11.02.04_RTSTR_n1__00000/2.16.840.1.114362.1.12289667.27454039548.698351266.280.551.dcm")
reader.LoadPrivateTagsOn()
reader.ReadImageInformation()
for k in reader.GetMetaDataKeys():
    v = reader.GetMetaData(k)
    print(f'({k}) = = "{v}"')

RuntimeError: Exception thrown in SimpleITK ImageFileReader_ReadImageInformation: /tmp/SimpleITK/Code/IO/src/sitkImageReaderBase.cxx:99:
sitk::ERROR: Unable to determine ImageIO reader for "/home/tachennf/Documents/delta-rad/madrid_study/data/Mont1/Montpellier_mont1_RTst_2024-07-11_110955_MRI.Study.from.2024-07-11.02.04_RTSTR_n1__00000/2.16.840.1.114362.1.12289667.27454039548.698351266.280.551.dcm"

# DicomRTTool

In [18]:
from DicomRTTool.ReaderWriter import DicomReaderWriter, ROIAssociationClass
Dicom_path = "/home/tachennf/Documents/delta-rad/madrid_study/data/Mont1/Montpellier_mont1_RTst_2024-07-11_110955_MRI.Study.from.2024-07-11.02.04_RTSTR_n1__00000"
Dicom_reader = DicomReaderWriter(description='Examples', arg_max=True)
Dicom_reader.walk_through_folders(Dicom_path) # This will parse through all DICOM present in the folder and subfolders
all_rois = Dicom_reader.return_rois(print_rois=True) # Return a list of all rois present


Loading through DICOM files: 100%|██████████| 1/1 [00:00<00:00, 11.47it/s]

Loading from /home/tachennf/Documents/delta-rad/madrid_study/data/Mont1/Montpellier_mont1_RTst_2024-07-11_110955_MRI.Study.from.2024-07-11.02.04_RTSTR_n1__00000
Compiling dictionaries together...
Index 0, description None at None
1 unique series IDs were found. Default is index 0, to change use set_index(index)
The following ROIs were found
